# Лабораторная работа 3

1. Определить две бизнес-цели для набора данных по Вашему варианту задания.

2. Определить цели технического проекта для каждой выделенной ранее бизнес-цели.

3. При необходимости следует использовать методы сбора и подготовки данных, которые были рассмотрены в лекции № 2.

4. Выполнить разбиение каждого набора данных на обучающую, контрольную и тестовую выборки для устранения проблемы просачивания данных.

5. Оценить сбалансированность выборок для каждого набора данных. Применить методы приращения (аугментации) данных при необходимости.

6. Выполнить процесс конструирования признаков для решения каждой из двух задач. Задача определяется совокупностью бизнес-цели и цели технического проекта.

7. При конструировании признаков обязательно необходимо выполнить: унитарное кодирование категориальных признаков (one-hot encoding), дискретизацию числовых признаков, «ручной» синтез признаков, масштабирование признаков на основе нормировки и стандартизации.

8. При необходимости выполнить конструирование признаков с применением фреймворка Featuretools (https://featuretools.alteryx.com/en/stable/).

9. Оценить качество каждого набора признаков по следующим критериям: предсказательная способность, скорость вычисления, надежность, корреляция и цельность.

10. Все выводы и программный код должны быть оформлены в виде ноутбука. Для выполнения данной лабораторной работы следует создать новый файл-ноутбук.


# Ход работы

## 1. Определить две бизнес-цели для набора данных по Вашему варианту задания.

В качестве датасета, были использованы данные о магазинах. Поставлены такие бизнес задачи
1) На основе данных найти зависимость большого кол-ва посетителей
2) На основе данных найти зависимость продаж

## 2. Определить цели технического проекта для каждой выделенной ранее бизнес-цели.

1) Оценить размеры и кол-во товаров магазина для понимания зависимости кол-ва посетителей
2) Оценить размеры и кол-во товаров магазина для понимания зависимости продаж

## 3. При необходимости следует использовать методы сбора и подготовки данных, которые были рассмотрены в лекции № 2.

## 4. Выполнить разбиение каждого набора данных на обучающую, контрольную и тестовую выборки для устранения проблемы просачивания данных.

Разбиение на выборки будут выполнены после дискретизации, тк данные плохо группируются. В целевых признаках нет просачивания.
Изначально планировалось в ключевые признаки ежедневных покупок, добавить кол-во людей, но эти признаки имеют целевую зависимость 

## 5. Оценить сбалансированность выборок для каждого набора данных. Применить методы приращения (аугментации) данных при необходимости.
## 6. Выполнить процесс конструирования признаков для решения каждой из двух задач. Задача определяется совокупностью бизнес-цели и цели технического проекта.

## 7. При конструировании признаков обязательно необходимо выполнить: унитарное кодирование категориальных признаков (one-hot encoding), дискретизацию числовых признаков, «ручной» синтез признаков, масштабирование признаков на основе нормировки и стандартизации.

One hot encoding не имеет смысла из за большого кол-ва вариантов

In [63]:
import numpy as np
import pandas as pd
from pathlib import Path

dataset_path = Path('../datasets/Stores.csv')
df = pd.read_csv(dataset_path, index_col="Store ID ")

Проведем равномерную дискретизацию для двух задач.

In [64]:
stores_area_discret_labels = []
stores_area_daily_customers = 6

bin_edges_stores_area = np.linspace(500, 3000, stores_area_daily_customers)
for max_edge in bin_edges_stores_area[1:]:
    stores_area_discret_labels.append(f" <= {int(max_edge)}")

bin_edges_stores_area, stores_area_discret_labels

(array([ 500., 1000., 1500., 2000., 2500., 3000.]),
 [' <= 1000', ' <= 1500', ' <= 2000', ' <= 2500', ' <= 3000'])

In [65]:
pd.concat([df["Store_Area"], pd.cut(df["Store_Area"], list(bin_edges_stores_area), labels=stores_area_discret_labels)], axis=1)

,Store_Area,Store_Area
Store ID,,
1,1659,<= 2000
2,1461,<= 1500
3,1340,<= 1500
4,1451,<= 1500
5,1770,<= 2000
...,...,...
892,1582,<= 2000
893,1387,<= 1500
894,1200,<= 1500


In [66]:
items_discret_labels = []
num_of_bins_items = 8

bin_edges_items = np.linspace(800, 3000, num_of_bins_items)

for max_edge in bin_edges_items[1:]:
    items_discret_labels.append(f" <= {int(max_edge)}")
    
bin_edges_items, items_discret_labels

(array([ 800.        , 1114.28571429, 1428.57142857, 1742.85714286,
        2057.14285714, 2371.42857143, 2685.71428571, 3000.        ]),
 [' <= 1114',
  ' <= 1428',
  ' <= 1742',
  ' <= 2057',
  ' <= 2371',
  ' <= 2685',
  ' <= 3000'])

In [67]:
pd.concat([df["Items_Available"], pd.cut(df["Items_Available"], list(bin_edges_items), labels=items_discret_labels)], axis=1)

,Items_Available,Items_Available
Store ID,,
1,1961,<= 2057
2,1752,<= 2057
3,1609,<= 1742
4,1748,<= 2057
5,2111,<= 2371
...,...,...
892,1910,<= 2057
893,1663,<= 1742
894,1436,<= 1742


In [68]:
df["Stores_Area_Discritisised"] = pd.cut(df["Store_Area"], list(bin_edges_stores_area), labels=stores_area_discret_labels)
df["Items_Available_Discritisised"] = pd.cut(df["Items_Available"], list(bin_edges_items), labels=items_discret_labels)

df

,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales,Stores_Area_Discritisised,Items_Available_Discritisised
Store ID,,,,,,
1,1659,1961,530,66490,<= 2000,<= 2057
2,1461,1752,210,39820,<= 1500,<= 2057
3,1340,1609,720,54010,<= 1500,<= 1742
4,1451,1748,620,53730,<= 1500,<= 2057
5,1770,2111,450,46620,<= 2000,<= 2371
...,...,...,...,...,...,...
892,1582,1910,1080,66390,<= 2000,<= 2057
893,1387,1663,850,82080,<= 1500,<= 1742
894,1200,1436,1060,76440,<= 1500,<= 1742


Ручной синтез не представляется возможным. 

Применим масштабирование (нормирование)

In [69]:
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()

df["Store_Area_Count_Norm"] = min_max_scaler.fit_transform(
    df["Store_Area"].to_numpy().reshape(-1, 1)
).reshape(df["Store_Area"].shape)

df["Items_Available_Norm"] = min_max_scaler.fit_transform(
    df["Items_Available"].to_numpy().reshape(-1, 1)
).reshape(df["Items_Available"].shape)

df

,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales,Stores_Area_Discritisised,Items_Available_Discritisised,Store_Area_Count_Norm,Items_Available_Norm
Store ID,,,,,,,,
1,1659,1961,530,66490,<= 2000,<= 2057,0.607978,0.593084
2,1461,1752,210,39820,<= 1500,<= 2057,0.471802,0.472622
3,1340,1609,720,54010,<= 1500,<= 1742,0.388583,0.390202
4,1451,1748,620,53730,<= 1500,<= 2057,0.464924,0.470317
5,1770,2111,450,46620,<= 2000,<= 2371,0.684319,0.679539
...,...,...,...,...,...,...,...,...
892,1582,1910,1080,66390,<= 2000,<= 2057,0.555021,0.563689
893,1387,1663,850,82080,<= 1500,<= 1742,0.420908,0.421326
894,1200,1436,1060,76440,<= 1500,<= 1742,0.292297,0.290490


Применим масштабирование (cтандартизацию)

In [70]:
standart_scaler = preprocessing.StandardScaler()

df["Store_Area_Count_Norm"] = standart_scaler.fit_transform(
    df["Store_Area"].to_numpy().reshape(-1, 1)
).reshape(df["Store_Area"].shape)

df["Items_Available_Norm_Standart"] = standart_scaler.fit_transform(
    df["Items_Available"].to_numpy().reshape(-1, 1)
).reshape(df["Items_Available"].shape)

df

,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales,Stores_Area_Discritisised,Items_Available_Discritisised,Store_Area_Count_Norm,Items_Available_Norm,Items_Available_Norm_Standart
Store ID,,,,,,,,,
1,1659,1961,530,66490,<= 2000,<= 2057,0.694091,0.593084,0.597135
2,1461,1752,210,39820,<= 1500,<= 2057,-0.097600,0.472622,-0.100218
3,1340,1609,720,54010,<= 1500,<= 1742,-0.581412,0.390202,-0.577354
4,1451,1748,620,53730,<= 1500,<= 2057,-0.137585,0.470317,-0.113564
5,1770,2111,450,46620,<= 2000,<= 2371,1.137919,0.679539,1.097628
...,...,...,...,...,...,...,...,...,...
892,1582,1910,1080,66390,<= 2000,<= 2057,0.386211,0.563689,0.426968
893,1387,1663,850,82080,<= 1500,<= 1742,-0.393485,0.421326,-0.397177
894,1200,1436,1060,76440,<= 1500,<= 1742,-1.141194,0.290490,-1.154589


## 8. При необходимости выполнить конструирование признаков с применением фреймворка Featuretools (https://featuretools.alteryx.com/en/stable/).

С учетом что ручного синтеза не было произведено, попробуем сделать это с помощью Featuretools

In [71]:
import featuretools as ft

pure_df = pd.read_csv(dataset_path, index_col="Store ID ")

es = ft.EntitySet(id="Store ID ")
es = es.add_dataframe(
    dataframe_name="stores",
    dataframe=pure_df,
    index="Store ID ",
)

es

/Users/shadowik/PycharmProjects/Data_Science_university_course/.venv/lib/python3.12/site-packages/featuretools/entityset/entityset.py:1733: UserWarning: index Store ID  not found in dataframe, creating new integer column
  warnings.warn(


Entityset: Store ID 
  DataFrames:
    stores [Rows: 896, Columns: 5]
  Relationships:
    No relationships

In [72]:
feature_matrix, feature_defs = ft.dfs(
    entityset=es,
    target_dataframe_name="stores",
    agg_primitives=["mean", "count", "mode", "any"],
    trans_primitives=["hour", "weekday"],
    max_depth=1,
)

feature_matrix

/Users/shadowik/PycharmProjects/Data_Science_university_course/.venv/lib/python3.12/site-packages/featuretools/synthesis/dfs.py:321: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  trans_primitives: ['hour', 'weekday']
  agg_primitives: ['any', 'count', 'mean', 'mode']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data. If the DFS call contained multiple instances of a primitive in the list above, none of them were used.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


,Store_Area,Items_Available,Daily_Customer_Count,Store_Sales
Store ID,,,,
0,1659,1961,530,66490
1,1461,1752,210,39820
2,1340,1609,720,54010
3,1451,1748,620,53730
4,1770,2111,450,46620
...,...,...,...,...
891,1582,1910,1080,66390
892,1387,1663,850,82080
893,1200,1436,1060,76440


Как и было сказано при попытке ручного синтеза, интересных данных из данного датасета получить невозможно


## 9. Оценить качество каждого набора признаков по следующим критериям: предсказательная способность, скорость вычисления, надежность, корреляция и цельность.

1) Предсказательная способность - Есть вариант что данных не достатотачно для точного прогнозирования.
2) Скорость вычисления - для работы не требуется большого кол-ва времени
3) Надежность - В данном контексте проблем нет, но параметр кол-во вещей, вполне может быть изменяемым
4) Кореляция - прямой кореляции не обнаружено
5) цельность - данные не зависят от других 